# ゲハ・スタイルレポート

In [128]:
# 標準ライブラリ
from datetime import datetime, timedelta

# サードパーティライブラリ
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from IPython.display import Markdown, display

# プロジェクト内モジュール
from gamedata import hard_sales as hs
from gamedata import plot_hard as ph

In [130]:
hard_sales_df = hs.load_hard_sales(normalize7=True)

## 歴代ハードとSwitch2の同時期のセールス状況比較

Switch2の最新発売週の状況を他の機種の同じ週と比較できる一覧表。 
- 例えば、deltaweekが8で絞り込んで、横軸にsumunits,units,ハード正式名称が並ぶ。
- 縦軸は累計で多い方からソートされる。
- Switch2の最新発売週は、hw==NS2で、deltaweekの最大値。

In [131]:
recent_week = hard_sales_df[hard_sales_df['hw'] == 'NS2'].sort_values('delta_week', ascending=False).head(1)['delta_week'].values[0]
recent_week

7

In [133]:
delta_recent_df = hard_sales_df[(hard_sales_df['delta_week'] == recent_week)]

# delta_recent_dfからカラム、full_name, sum_units, unitsを抽出
delta_recent_df = delta_recent_df[['full_name', 'sum_units', 'units']]
# sum_units, unitsで降順ソート
delta_recent_df = delta_recent_df.sort_values(by=['sum_units', 'units'], ascending=False)
display(Markdown(f'## 歴代ハードウェアの{recent_week + 1}週目の販売台数比較'))

# カラム名を日本語に差し替える
delta_recent_df.rename(columns={
    'full_name': 'ハードウェア名',
    'sum_units': '累計販売台数',
    'units': '最新週販売台数'
}, inplace=True)
delta_recent_df.set_index('ハードウェア名', inplace=True)
delta_recent_df.style.format('{:,}') 

## 歴代ハードウェアの8週目の販売台数比較

,累計販売台数,最新週販売台数
ハードウェア名,,
Nintendo Switch2,"1,846,889","92,013"
GAME BOY ADVANCE,"1,738,487","88,872"
Nintendo DS,"1,508,241","47,291"
PlayStation2,"1,404,366","74,350"
Wii,"1,313,160","80,514"
Nintendo 3DS,"902,680","25,320"
WiiU,"759,460","12,335"
PlayStation Portable,"701,680","71,200"
Nintendo Switch,"684,971","40,367"


## 直近4週間のハード売上／累計推移

In [ ]:
# hard_sales_df から、今週〜4週間前までの units とsum_unitsのデータを抽出
# 横軸に report_date,  full_name, units, sum_units
# report_date, full_nameは　階層indexとする。

last_report_date = hard_sales_df['report_date'].iloc[-1]  # 最新のデータの日付を取得
from pandas import Timedelta
start_date = last_report_date - Timedelta(weeks=3)  # 4週間前の日付を計算


last4weeks_df = hard_sales_df[hard_sales_df['report_date'] >= start_date][['report_date', 'full_name', 'units', 'sum_units']]
last4weeks_df.set_index(['report_date', 'full_name'], inplace=True)
last4weeks_df.sort_values(by=['report_date', 'units', 'full_name'], ascending=[True, False, True], inplace=True)

# カラム名を日本語にする
last4weeks_df.columns = ['販売数', '累計']
# indexを日本語にする
last4weeks_df.index.names = ['報告日', 'ハード']

last4weeks_df.index = last4weeks_df.index.set_levels(
    last4weeks_df.index.levels[0].strftime('%Y-%m-%d'), level=0
)
# スタイルを適用して表示
last4weeks_df.style.format({'販売数': '{:,}', '累計': '{:,}'})


## Switch2とPSのロンチ直後の売上比較

In [119]:
ns2_ps5_df = hs.pivot_sales_by_delta(hard_sales_df, hw=['NS2', 'PS5'], full_name=True)

# 列のどれかにNaNがあったらdrop
ns2_ps5_df = ns2_ps5_df.dropna()
ns2_ps5_df.index = [f'{i+1}週' for i in ns2_ps5_df.index]

# 各カラムの全行を合計して、累計行を末尾に追加
ns2_ps5_df.loc['累計'] = ns2_ps5_df.sum(numeric_only=True)

ns2_ps5_df.index.name = '週'

ns2_ps5_df.style.format('{:,.0f}') 

full_name,Nintendo Switch2,PlayStation5
週,,
1週,"947,931","118,085"
2週,"153,205","42,891"
3週,"140,026","40,459"
4週,"161,021","11,893"
5週,"128,643","11,056"
6週,"71,885","17,578"
7週,"152,165","13,188"
8週,"92,013","10,632"
累計,"1,846,889","265,782"


In [ ]:
ns2_all = ns2_ps5_df["Nintendo Switch2"].iloc[-1]
ps5_all = ns2_ps5_df["PlayStation5"].iloc[-1]

diff_ns2ps5 = ns2_all - ps5_all

display(Markdown(f'## NS2とPS5の累計販売台数の差: {diff_ns2ps5:,.0f}台'))

## NS2とPS5の累計販売台数の差: 1,581,107台

In [138]:
a_df = hs.extract_week_reached_units(hard_sales_df, threshold_units=ns2_all)
# a_dfからhw名がPS5の行を抽出
ps5_df = a_df[a_df['hw'] == 'PS5']
ps5_df['delta_week']

a_df

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_month,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
4394,2011-09-18_3DS,2011-09-12,2011-09-18,2011-09-18,7,3DS,62245,2011,9,18,37,204,29,7,0,8892,1849714,2011-02-26,Nintendo,Nintendo 3DS
1033,2001-06-24_DC,2001-06-18,2001-06-24,2001-06-24,7,DC,5697,2001,6,24,25,940,134,31,3,813,1849653,1998-11-27,SEGA,DreamCast
2249,2005-04-03_DS,2005-03-28,2005-04-03,2005-04-03,7,DS,43806,2005,4,3,13,122,17,4,1,6258,1853112,2004-12-02,Nintendo,Nintendo DS
128,1999-03-28_GB,1999-03-22,1999-03-28,1999-03-28,7,GB,40726,1999,3,28,12,3628,518,119,10,5818,1876502,1989-04-21,Nintendo,GAME BOY
990,2001-05-27_GBA,2001-05-21,2001-05-27,2001-05-27,7,GBA,71334,2001,5,27,21,67,9,2,0,10190,1880696,2001-03-21,Nintendo,GAME BOY ADVANCE
1613,2002-12-22_GC,2002-12-16,2002-12-22,2002-12-22,7,GC,66410,2002,12,22,51,464,66,15,1,9487,1900120,2001-09-14,Nintendo,Nintendo GAMECUBE
8439,2025-07-27_NS2,2025-07-21,2025-07-27,2025-07-27,7,NS2,92013,2025,7,27,30,52,7,1,0,13144,1846889,2025-06-05,Nintendo,Nintendo Switch2
6492,2017-10-08_NSW,2017-10-02,2017-10-08,2017-10-08,7,NSW,38425,2017,10,8,40,219,31,7,0,5489,1847008,2017-03-03,Nintendo,Nintendo Switch
309,1999-09-26_PKS,1999-09-20,1999-09-26,1999-09-26,7,PKS,77811,1999,9,26,38,246,35,8,0,11115,1911732,1999-01-23,SONY,PocketStation
249,1999-07-25_PS,1999-07-19,1999-07-25,1999-07-25,7,PS,37650,1999,7,25,29,1695,242,55,5,5378,1865110,1994-12-03,SONY,PlayStation


In [ ]:
last4weeks_df = last4weeks_df[['report_date', 'full_name', 'units', 'sum_units']]
last4weeks_df.set_index(['report_date', 'full_name'], inplace=True)
last4weeks_df = last4weeks_df.sort_index(level='full_name')
last4weeks_df = last4weeks_df.unstack(level='full_name')
last4weeks_df.columns = last4weeks_df.columns.droplevel(0)

last4weeks_df

In [38]:
last4weeks_df

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_month,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8424,2025-07-06_NS2,2025-06-30,2025-07-06,2025-07-06,7,NS2,128643,2025,7,6,27,31,4,1,0,18377,1530826,2025-06-05,Nintendo,Nintendo Switch2
8425,2025-07-06_NSW,2025-06-30,2025-07-06,2025-07-06,7,NSW,17712,2025,7,6,27,3047,435,100,8,2530,35811161,2017-03-03,Nintendo,Nintendo Switch
8426,2025-07-06_PS4,2025-06-30,2025-07-06,2025-07-06,7,PS4,21,2025,7,6,27,4152,593,137,11,3,9490618,2014-02-22,SONY,PlayStation4
8427,2025-07-06_PS5,2025-06-30,2025-07-06,2025-07-06,7,PS5,8629,2025,7,6,27,1697,242,56,5,1232,6918908,2020-11-12,SONY,PlayStation5
8428,2025-07-06_XSX,2025-06-30,2025-07-06,2025-07-06,7,XSX,529,2025,7,6,27,1699,242,56,5,75,681127,2020-11-10,Microsoft,Xbox Series X|S
8429,2025-07-13_XSX,2025-07-07,2025-07-13,2025-07-13,7,XSX,247,2025,7,13,28,1706,243,56,5,35,681374,2020-11-10,Microsoft,Xbox Series X|S
8430,2025-07-13_PS5,2025-07-07,2025-07-13,2025-07-13,7,PS5,7262,2025,7,13,28,1704,243,56,5,1037,6926170,2020-11-12,SONY,PlayStation5
8431,2025-07-13_PS4,2025-07-07,2025-07-13,2025-07-13,7,PS4,30,2025,7,13,28,4159,594,137,11,4,9490648,2014-02-22,SONY,PlayStation4
8432,2025-07-13_NS2,2025-07-07,2025-07-13,2025-07-13,7,NS2,71885,2025,7,13,28,38,5,1,0,10269,1602711,2025-06-05,Nintendo,Nintendo Switch2
8433,2025-07-13_NSW,2025-07-07,2025-07-13,2025-07-13,7,NSW,13411,2025,7,13,28,3054,436,100,8,1915,35824572,2017-03-03,Nintendo,Nintendo Switch
